In [1]:
import boto3
import s3fs
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics

In [2]:
client=boto3.client('s3')
path="s3://eg275s-group1-data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_1.csv"

In [3]:
df=pd.read_csv(path, low_memory=False)

In [4]:
df.shape

(570118, 110)

In [5]:
print(f'Rows and columns in one CSV file is {df.shape}')

Rows and columns in one CSV file is (570118, 110)


In [6]:
listcolumns = list(df)

In [7]:
print(listcolumns)

['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Reporting_Airline', 'DOT_ID_Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime'

In [8]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2018,1,1,27,6,2018-01-27,UA,19977,UA,N26232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,1,1,27,6,2018-01-27,UA,19977,UA,N477UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,1,1,27,6,2018-01-27,UA,19977,UA,N13720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,1,1,27,6,2018-01-27,UA,19977,UA,N16217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,1,1,27,6,2018-01-27,UA,19977,UA,N33714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Filter Origin by just one airport - JFK (John F. Kennedy International Airport)
val=['JFK']
jfk_df = df[df['Origin'].isin(val)]

In [10]:
jfk_df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
45390,2018,1,1,1,1,2018-01-01,AS,19930,AS,N277AK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45629,2018,1,1,1,1,2018-01-01,AS,19930,AS,N520AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45893,2018,1,1,2,2,2018-01-02,AS,19930,AS,N282AK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46150,2018,1,1,2,2,2018-01-02,AS,19930,AS,N557AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46413,2018,1,1,3,3,2018-01-03,AS,19930,AS,N281AK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
jfk_df.shape

(10050, 110)

In [12]:
# View the unique values in the column
jfk_df['Dest'].unique()

array(['SEA', 'PDX', 'BOS', 'CLE', 'CLT', 'DTW', 'DCA', 'IAD', 'ROC',
       'PHL', 'RIC', 'BNA', 'PWM', 'BWI', 'MSY', 'CMH', 'BUF', 'TPA',
       'CVG', 'SAV', 'SYR', 'SRQ', 'RDU', 'BGR', 'CHS', 'PIT', 'MSP',
       'ORD', 'DFW', 'IND', 'JAX', 'ORF', 'FLL', 'SJU', 'BTV', 'ABQ',
       'SLC', 'RNO', 'SAN', 'DEN', 'LAX', 'PHX', 'PSP', 'PBI', 'SMF',
       'OAK', 'LGB', 'MCO', 'RSW', 'BUR', 'DAB', 'SFO', 'HOU', 'SJC',
       'LAS', 'PSE', 'AUS', 'BQN', 'HNL', 'MIA', 'EGE', 'SAT', 'ATL',
       'JAC'], dtype=object)

In [13]:
jfk_df['OriginAirportID'].unique()

array([12478])

In [14]:
# preview data and show all columns
pd.options.display.max_columns = None
jfk_df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
45390,2018,1,1,1,1,2018-01-01,AS,19930,AS,N277AK,7,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,715,719.0,4.0,4.0,0.0,0.0,0700-0759,18.0,737.0,1008.0,16.0,1045,1024.0,-21.0,0.0,0.0,-2.0,1000-1059,0.0,NaN,0.0,390.0,365.0,331.0,1.0,2422.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45629,2018,1,1,1,1,2018-01-01,AS,19930,AS,N520AS,479,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,14057,1405702,34057,PDX,"Portland, OR",OR,41,Oregon,92,1030,1022.0,-8.0,0.0,0.0,-1.0,1000-1059,20.0,1042.0,1316.0,4.0,1356,1320.0,-36.0,0.0,0.0,-2.0,1300-1359,0.0,NaN,0.0,386.0,358.0,334.0,1.0,2454.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45893,2018,1,1,2,2,2018-01-02,AS,19930,AS,N282AK,7,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,715,716.0,1.0,1.0,0.0,0.0,0700-0759,30.0,746.0,1025.0,8.0,1045,1033.0,-12.0,0.0,0.0,-1.0,1000-1059,0.0,NaN,0.0,390.0,377.0,339.0,1.0,2422.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46150,2018,1,1,2,2,2018-01-02,AS,19930,AS,N557AS,479,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,14057,1405702,34057,PDX,"Portland, OR",OR,41,Oregon,92,1030,1027.0,-3.0,0.0,0.0,-1.0,1000-1059,25.0,1052.0,1344.0,4.0,1356,1348.0,-8.0,0.0,0.0,-1.0,1300-1359,0.0,NaN,0.0,386.0,381.0,352.0,1.0,2454.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46413,2018,1,1,3,3,2018-01-03,AS,19930,AS,N281AK,7,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,715,724.0,9.0,9.0,0.0,0.0,0700-0759,40.0,804.0,1045.0,9.0,1045,1054.0,9.0,9.0,0.0,0.0,1000-1059,0.0,NaN,0.0,390.0,390.0,341.0,1.0,2422.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [15]:
# drop columns not needed
jfk_df = jfk_df.drop(['Quarter', 'DayOfWeek', 'FlightDate', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportSeqID', 'OriginCityMarketID',  'OriginStateFips', 'OriginWac', 'DestAirportSeqID', 'DestCityMarketID', 'DestStateFips', 'DestWac', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup', 'CarrierDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime', 'DivAirportLandings', 'DivReachedDest', 'DivActualElapsedTime', 'DivArrDelay', 'DivDistance', 'Div1Airport', 'Div1AirportID', 'Div1AirportSeqID', 'Div1WheelsOn', 'Div1TotalGTime', 'Div1LongestGTime', 'Div1WheelsOff', 'Div1TailNum', 'Div2Airport', 'Div2AirportID', 'Div2AirportSeqID', 'Div2WheelsOn', 'Div2TotalGTime', 'Div2LongestGTime', 'Div2WheelsOff', 'Div2TailNum', 'Div3Airport', 'Div3AirportID', 'Div3AirportSeqID', 'Div3WheelsOn', 'Div3TotalGTime', 'Div3LongestGTime', 'Div3WheelsOff', 'Div3TailNum', 'Div4Airport', 'Div4AirportID', 'Div4AirportSeqID', 'Div4WheelsOn', 'Div4TotalGTime', 'Div4LongestGTime', 'Div4WheelsOff', 'Div4TailNum', 'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID', 'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff', 'Div5TailNum', 'Unnamed: 109'
], axis=1)

In [16]:
# View the unique values in the WeatherDelay column
a = jfk_df['WeatherDelay'].unique()
print(sorted(a))

[nan, 0.0, 2.0, 3.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 69.0, 72.0, 74.0, 77.0, 79.0, 82.0, 83.0, 85.0, 86.0, 87.0, 89.0, 90.0, 91.0, 92.0, 94.0, 96.0, 97.0, 99.0, 102.0, 103.0, 104.0, 106.0, 108.0, 112.0, 114.0, 118.0, 121.0, 122.0, 123.0, 125.0, 130.0, 135.0, 138.0, 139.0, 140.0, 145.0, 146.0, 155.0, 158.0, 159.0, 160.0, 161.0, 164.0, 174.0, 178.0, 180.0, 189.0, 193.0, 196.0, 197.0, 198.0, 220.0, 224.0, 234.0, 239.0, 240.0, 245.0, 246.0, 253.0, 270.0, 301.0, 304.0, 314.0, 315.0, 325.0, 350.0, 354.0, 375.0, 376.0, 379.0, 521.0, 700.0, 701.0]


In [17]:
jfk_df['WeatherDelay'].isnull().values.any()

True

In [18]:
jfk_df['WeatherDelay'].isnull().sum()

8190

In [19]:
jfk_df.columns[jfk_df.isnull().any()]

Index(['DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'ArrTime',
       'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'WeatherDelay'],
      dtype='object')

In [20]:
jfk_df.head()

,Year,Month,DayofMonth,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,WeatherDelay
45390,2018,1,1,AS,19930,AS,12478,JFK,"New York, NY",NY,New York,14747,SEA,"Seattle, WA",WA,Washington,715,719.0,4.0,4.0,0.0,1045,1024.0,-21.0,0.0,0.0,NaN
45629,2018,1,1,AS,19930,AS,12478,JFK,"New York, NY",NY,New York,14057,PDX,"Portland, OR",OR,Oregon,1030,1022.0,-8.0,0.0,0.0,1356,1320.0,-36.0,0.0,0.0,NaN
45893,2018,1,2,AS,19930,AS,12478,JFK,"New York, NY",NY,New York,14747,SEA,"Seattle, WA",WA,Washington,715,716.0,1.0,1.0,0.0,1045,1033.0,-12.0,0.0,0.0,NaN
46150,2018,1,2,AS,19930,AS,12478,JFK,"New York, NY",NY,New York,14057,PDX,"Portland, OR",OR,Oregon,1030,1027.0,-3.0,0.0,0.0,1356,1348.0,-8.0,0.0,0.0,NaN
46413,2018,1,3,AS,19930,AS,12478,JFK,"New York, NY",NY,New York,14747,SEA,"Seattle, WA",WA,Washington,715,724.0,9.0,9.0,0.0,1045,1054.0,9.0,9.0,0.0,NaN


In [21]:
jfk_df.shape

(10050, 27)

In [22]:
jfk_df.dropna(subset=['WeatherDelay'], inplace=True)

In [23]:
jfk_df.shape

(1860, 27)

In [24]:
# List columns with nan
jfk_df.columns[jfk_df.isnull().any()]

Index(['DepDelay', 'DepDelayMinutes', 'DepDel15'], dtype='object')

In [25]:
# View the unique values in the WeatherDelay column
a = jfk_df['WeatherDelay'].unique()
print(sorted(a))

[0.0, 2.0, 3.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 69.0, 72.0, 74.0, 77.0, 79.0, 82.0, 83.0, 85.0, 86.0, 87.0, 89.0, 90.0, 91.0, 92.0, 94.0, 96.0, 97.0, 99.0, 102.0, 103.0, 104.0, 106.0, 108.0, 112.0, 114.0, 118.0, 121.0, 122.0, 123.0, 125.0, 130.0, 135.0, 138.0, 139.0, 140.0, 145.0, 146.0, 155.0, 158.0, 159.0, 160.0, 161.0, 164.0, 174.0, 178.0, 180.0, 189.0, 193.0, 196.0, 197.0, 198.0, 220.0, 224.0, 234.0, 239.0, 240.0, 245.0, 246.0, 253.0, 270.0, 301.0, 304.0, 314.0, 315.0, 325.0, 350.0, 354.0, 375.0, 376.0, 379.0, 521.0, 700.0, 701.0]


In [26]:
# preview the dataset
jfk_df.shape

(1860, 27)

In [27]:
# View the unique values in the column
jfk_df['Dest'].unique()

array(['SEA', 'PDX', 'BOS', 'DCA', 'IAD', 'ROC', 'PWM', 'MSY', 'BUF',
       'TPA', 'BWI', 'CVG', 'SAV', 'SYR', 'DTW', 'SRQ', 'BNA', 'BGR',
       'PHL', 'RIC', 'RDU', 'PIT', 'ORD', 'DFW', 'CHS', 'IND', 'JAX',
       'CLE', 'CLT', 'ORF', 'MSP', 'CMH', 'FLL', 'SJU', 'BTV', 'LAX',
       'PBI', 'SAN', 'MCO', 'DAB', 'SFO', 'RSW', 'AUS', 'BQN', 'SLC',
       'DEN', 'HOU', 'SJC', 'PSE', 'ABQ', 'RNO', 'PHX', 'PSP', 'SMF',
       'OAK', 'LAS', 'LGB', 'BUR', 'HNL', 'MIA', 'EGE', 'ATL', 'SAT'],
      dtype=object)

In [28]:
# filter by 5 destination airports
val=['LAS','SFO','DEN','MCO','DCA']
filtered_df = jfk_df[jfk_df['Dest'].isin(val)]

# LAS, 12889, NEVADA (NV), Las Vegas, Harry Reid International Airport
# SFO, 14771, CALIFORNIA (CA), San Francisco, San Francisco International Airport
# DEN, 11292, Colorado (CO), Denver, Denver International Airport
# MCO, 13204, FLORIDA (FL), Orlando, Orlando International Airport
# DCA, 11278, VIRGINIA, Washington, D.C. / Arlington, Ronald Reagan Washington National Airport

In [29]:
filtered_df.head()

,Year,Month,DayofMonth,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,WeatherDelay
68418,2018,1,5,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,2100,131.0,271.0,271.0,1.0,2232,230.0,238.0,238.0,1.0,0.0
68419,2018,1,6,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,2100,2150.0,50.0,50.0,1.0,2230,2259.0,29.0,29.0,1.0,29.0
68430,2018,1,17,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,2059,2149.0,50.0,50.0,1.0,2231,2255.0,24.0,24.0,1.0,0.0
68442,2018,1,29,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,2059,716.0,617.0,617.0,1.0,2231,906.0,635.0,635.0,1.0,0.0
68835,2018,1,8,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,1130,1444.0,194.0,194.0,1.0,1258,1707.0,249.0,249.0,1.0,0.0


In [30]:
filtered_df.shape

(299, 27)

In [31]:
filtered_df['DestAirportID'].unique()

array([11278, 13204, 14771, 11292, 12889])

In [32]:
filtered_df['DestState'].unique()

array(['VA', 'FL', 'CA', 'CO', 'NV'], dtype=object)

In [33]:
filtered_df.head()

,Year,Month,DayofMonth,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,WeatherDelay
68418,2018,1,5,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,2100,131.0,271.0,271.0,1.0,2232,230.0,238.0,238.0,1.0,0.0
68419,2018,1,6,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,2100,2150.0,50.0,50.0,1.0,2230,2259.0,29.0,29.0,1.0,29.0
68430,2018,1,17,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,2059,2149.0,50.0,50.0,1.0,2231,2255.0,24.0,24.0,1.0,0.0
68442,2018,1,29,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,2059,716.0,617.0,617.0,1.0,2231,906.0,635.0,635.0,1.0,0.0
68835,2018,1,8,9E,20363,9E,12478,JFK,"New York, NY",NY,New York,11278,DCA,"Washington, DC",VA,Virginia,1130,1444.0,194.0,194.0,1.0,1258,1707.0,249.0,249.0,1.0,0.0


In [34]:
print(list(filtered_df))

['Year', 'Month', 'DayofMonth', 'Reporting_Airline', 'DOT_ID_Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'OriginAirportID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateName', 'DestAirportID', 'Dest', 'DestCityName', 'DestState', 'DestStateName', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'WeatherDelay']


In [35]:
# 4. Feature and Target Selection

In [36]:
# Define features and target variable
features = ['Year', 'Month', 'DayofMonth', 'OriginAirportID', 'DestAirportID', 'DepTime']
target = 'WeatherDelay'  # this is what we want to predict based on the data in features

In [37]:
X = filtered_df[features].values
y = filtered_df[target].values

In [38]:
# 5. Splitting Data into Training and Test Sets

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [40]:
# Look at the shape of X_train and X_test
X_train.shape, X_test.shape

((239, 6), (60, 6))

In [41]:
# 6. Standardize the Data
# Logistic Regression is effected by scale so you need to scale the features.

In [42]:
# Verify the Origin is only one airport
filtered_df['Origin'].nunique()

1

In [43]:
# Verify the Origin is JFK International Airport
filtered_df['Origin'].unique()

array(['JFK'], dtype=object)

In [44]:
# Splitting Train-set and Test-set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2
                                                    , random_state=41)

# Splitting Train-set and Validation-set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.25,
                                                  random_state=41)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

# Serialize the data
train_data = {'X_train': X_train, 'y_train': y_train}
test_data = {'X_test': X_test, 'y_test': y_test}

(179, 6) (60, 6)
(179,) (60,)


In [45]:
import boto3
from sagemaker import image_uris

region = boto3.client('ec2').meta.region_name
container = image_uris.retrieve('linear-learner', region)
print(container)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
469771592824.dkr.ecr.ca-central-1.amazonaws.com/linear-learner:1


In [46]:
import pickle
import time
from io import BytesIO
from sklearn.model_selection import train_test_split, GridSearchCV


# Store train and test data in S3
s3 = boto3.resource('s3')
bucket_name = 'eg275s-group1-data-central'
prefix = 'test-train_data'

# Storing train data
train_data_bytes = BytesIO()
pickle.dump(train_data, train_data_bytes)
train_data_bytes.seek(0)
s3.Object(bucket_name, f'{prefix}/train_data.pkl').put(Body=train_data_bytes)

# Storing test data
test_data_bytes = BytesIO()
pickle.dump(test_data, test_data_bytes)
test_data_bytes.seek(0)
s3.Object(bucket_name, f'{prefix}/test_data.pkl').put(Body=test_data_bytes)

# Set an S3 output path where the trained model will be saved
output_path = f's3://{bucket_name}/{prefix}/output'
print(output_path)

s3://eg275s-group1-data-central/test-train_data/output


In [47]:
import sagemaker
from sagemaker.inputs import TrainingInput

# Retrieve the notebooks role and use it for model training (has access to S3, SageMaker)
role = sagemaker.get_execution_role()

# Create a SageMaker session
session = sagemaker.Session()

# Use the Estimator interface to configure a training job with the linear learner algorithm
linear_learner = sagemaker.estimator.Estimator(container, role, instance_count=1, instance_type='ml.m4.xlarge', output_path=output_path, sagemaker_session=session)


In [48]:
print(boto3.client('sagemaker').meta.region_name)

ca-central-1


In [49]:
print(df.columns)

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'DOT_ID_Reporting_Airline',
       'IATA_CODE_Reporting_Airline', 'Tail_Number',
       ...
       'Div4TailNum', 'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID',
       'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff',
       'Div5TailNum', 'Unnamed: 109'],
      dtype='object', length=110)


In [50]:
# Define the S3 location for training and output data
train_data_uri = f's3://{bucket_name}/{prefix}/train_data.pkl'
test_data_uri = f's3://{bucket_name}/{prefix}/test_data.pkl'

# Set hyperparameters
linear_learner.set_hyperparameters(feature_dim=X_train.shape[1],
                                    mini_batch_size=100,
                                    predictor_type='binary_classifier')

# Define the data channels for training
train_channel = TrainingInput(train_data_uri, content_type='application/x-recordio-protobuf')
test_channel = TrainingInput(test_data_uri, content_type='application/x-recordio-protobuf')

print(train_channel)
print(train_channel.__dict__)




{'config': {'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://eg275s-group1-data-central/test-train_data/train_data.pkl', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'application/x-recordio-protobuf'}}


In [ ]:
# Create the training job
linear_learner.fit({'train': train_channel, 'test': test_channel})